Import important libraries first!

In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys
!{sys.executable} -m pip install geocoder
import geocoder

    100% |████████████████████████████████| 102kB 19.7MB/s 
distributed 1.21.8 requires msgpack, which is not installed.


Getting the xml from Wikipedia and processing it using the BeautifulSoup Library

In [36]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Extracting the table element from the Wikipedia page and creating the dictionary to create the DataFrame later.

In [37]:
table = soup.find('table').tbody
table_rows = table.find_all('tr')
data_dict = {'PostalCode':[],'Borough':[],'Neighborhood':[],'Latitude':[],'Longitude':[]}


Processing the rows so that the Borough that are not assigned are ignored and the neighborhoods that are not assigned copy from the bourough.
Then for each row the processed information is inserted to the dictionary

In [ ]:
for row in table_rows:
    elements = row.find_all('td')
    if len(elements)==0:
        continue
    if elements[1].string == "Not assigned" and elements[2].string=="Not assigned\n":
        continue
    elif elements[1].text != 'Not assigned' and elements[2].text=='Not assigned\n':
        elements[2].string = elements[1].text
    lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(elements[0].string))
    
      lat_lng_coords = g.latlng
    data_dict["Latitude"].append(lat_lng_coords[0])
    data_dict["Longitude"].append(lat_lng_coords[1])
    data_dict['PostalCode'].append(elements[0].string)
    data_dict['Borough'].append(elements[1].string)
    data_dict['Neighborhood'].append(elements[2].text.strip()+",")    

The final dictionary is turned into a pandas DataFrame then the groupby method is called so that duplicate postalcodes are consolidated.
Then sun concatenates the neighborhoods giving the final dataframe

In [ ]:
data_df = pd.DataFrame.from_dict(data_dict)
data_df = data_df.groupby(["PostalCode","Borough"],as_index=False).sum()

This gives the shape of the final DataFrame and the head of it

In [ ]:
print(data_df.shape)
data_df.head(10)